In [ ]:
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Json"
#r "nuget: Plotly.NET.ImageExport"

open Plotly.NET
open Plotly.NET.ImageExport

#load "Helpers.fs"
#load "Bestiary.fs"
#load "Compare.fs"
#load "Library.fs"
#load "Transform.fs"

open PathfinderAnalysis.Library
open PathfinderAnalysis.Bestiary
open PathfinderAnalysis.Compare
open PathfinderAnalysis.Transform
open PathfinderAnalysis.Helpers

In [ ]:
[ Chart.Line(
    x = pcLevels,
    y = List.map (highMartialAttack true) pcLevels,
    Name = "Martial Attack Proficiency",
    ShowMarkers = true
);
Chart.Line(
    x = pcLevels,
    y = List.map (casterAttack true false) pcLevels,
    Name = "Caster Attack Proficiency",
    ShowMarkers = true
)]
|> Chart.combine
|> Chart.withLineStyle(Shape = StyleParam.Shape.Hvh)
|> Chart.withXAxisStyle(TitleText = "Character Level", MinMax = (0, 21))
|> Chart.withYAxisStyle(TitleText = "Attack Bonus")
|> Chart.withTitle "Martial-Caster Attack Disparity"


In [12]:
let titleFn level = 
  sprintf "Middle Save - Level %i - Spout Cantrip vs. Martial Arbalest Against Level %i Creatures" level (level+2)

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel spout CreatureSave middleSave (casterDc true) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Spout"
  };
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel martialArbalest PlayerAttack creatureAc (highMartialAttack true) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "Arbalest"
  }
|]
|> flatten
|> generateCharts titleFn
|> Seq.iteri (fun i chart -> Chart.savePNG (path = (sprintf "spout-arbalest-%02i" i)) chart)
|> ignore